Modified code from Chapter 10 of Machine Learning: An Algorithmic Perspective by Stephen Marsland (http://seat.massey.ac.nz/personal/s.r.marsland/MLBook.html)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

In [ ]:
def swissroll():
        # Make the swiss roll dataset
        N = 1000
        noise = 0.05

        t = 3*np.math.pi/2 * (1 + 2*np.random.rand(1,N))
        h = 21 * np.random.rand(1,N)
        data = np.concatenate((t*np.cos(t),h,t*np.sin(t))) + noise*np.random.randn(3,N)
        return data.T, np.squeeze(t)
    
data,t = swissroll()
fig = plt.figure()
ax = Axes3D(fig)
ax.scatter3D(data[:,0],data[:,1],data[:,2],s=50,c=t)
plt.show()

In [ ]:
def LLE(data,nRedDim=2,K=12):

    ndata = data.shape[0]
    ndim = data.shape[1]
    d = np.zeros((ndata,ndata),dtype=float)

    # Inefficient -- not matrices
    for i in range(ndata):
        for j in range(i+1,ndata):
            for k in range(ndim):
                d[i,j] += (data[i,k] - data[j,k])**2
            d[i,j] = np.sqrt(d[i,j])
            d[j,i] = d[i,j]

    indices = d.argsort(axis=1)
    neighbours = indices[:,1:K+1]

    W = np.zeros((K,ndata),dtype=float)

    for i in range(ndata):
        Z  = data[neighbours[i,:],:] - np.kron(np.ones((K,1)),data[i,:])
        C = np.dot(Z,Z.T)
        C = C+np.identity(K)*1e-3*np.trace(C)
        W[:,i] = np.transpose(np.linalg.solve(C,np.ones((K,1))))
        W[:,i] = W[:,i]/sum(W[:,i])
    
    M = np.eye(ndata,dtype=float)
    for i in range(ndata):
        w = np.transpose(np.ones((1,W.shape[0]))*np.transpose(W[:,i]))
        j = neighbours[i,:]
        #print shape(w), shape(dot(w,transpose(w))), shape(M[i,j])
        ww = np.dot(w,w.T)
        for k in range(K):
            M[i,j[k]] -= w[k]
            M[j[k],i] -= w[k]
            for l in range(K):
                 M[j[k],j[l]] += ww[k,l]

    evals,evecs = np.linalg.eig(M)
    ind = np.argsort(evals)
    y = evecs[:,ind[1:nRedDim+1]]*np.sqrt(ndata)
    return evals,evecs,y


In [ ]:
data,t = swissroll()
evals,evecs,y = LLE(data)

t -= t.min()
t /= t.max()
plt.scatter(y[:,0],y[:,1],s=50,c=t)
plt.show()